In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

# Load your data (using the corrected method to combine dates)
dft = pd.read_csv('../data/ubc_temperature91_02cl.csv')
dft['Date_Time'] = pd.to_datetime(dft['Date'] + ' ' + dft['Time'])
dft.drop(['Date', 'Time'], axis=1, inplace=True)
dft.set_index('Date_Time', inplace=True)

# Function to calculate the number of days below the cutoff temperature
def get_period(start, end, cutoff):
    dft1 = dft[start:end]
    # Number of hours below the cutoff temperature
    below_hours = (dft1['Tair'] <= cutoff).sum()
    # Convert hours to days
    below_days = round(below_hours / 24)
    return below_days

# Interactive widgets with default values
min_date = dft.index.min().date()
max_date = dft.index.max().date()

start_date = widgets.DatePicker(description='Start Date', value=min_date)
end_date = widgets.DatePicker(description='End Date', value=max_date)
temp_cutoff = widgets.FloatText(description='Temp Cutoff (°C)', value=0.0)
output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        # Check if dates are selected
        if not start_date.value or not end_date.value:
            print("Please select both start and end dates.")
            return
        # Ensure dates are strings in the correct format
        start = start_date.value.strftime('%Y-%m-%d')
        end = end_date.value.strftime('%Y-%m-%d')
        # Check if start date is before end date
        if start_date.value > end_date.value:
            print("Start date must be before end date.")
            return
        cutoff = temp_cutoff.value
        # Call the get_period function
        below_days = get_period(start, end, cutoff)
        # Display the result
        print(f"**Temperature Analysis**")
        print(f"Period: {start} to {end}")
        print(f"Temperature Cutoff: {cutoff}°C")
        print(f"Days Below Cutoff: {below_days} days")
        
        # Plotting the temperature over the period
        dft1 = dft[start:end]
        if dft1.empty:
            print("No data available for the selected period.")
            return
        plt.figure(figsize=(12, 6))
        plt.plot(dft1.index, dft1['Tair'], label='Temperature', color='blue')
        plt.axhline(y=cutoff, color='red', linestyle='--', label=f'Cutoff ({cutoff}°C)')
        plt.xlabel('Date')
        plt.ylabel('Temperature (°C)')
        plt.title('Temperature Over Selected Period')
        plt.legend()
        plt.gcf().autofmt_xdate()
        plt.show()
    
# Button to trigger the calculation
calculate_button = widgets.Button(description='Calculate', button_style='success')
calculate_button.on_click(on_button_click)

# Display widgets with a description
intro_text = widgets.HTML(
    value="<h3>Temperature Analysis Tool</h3><p>Select a date range and temperature cutoff to analyze the number of days below the specified temperature. The plot will show the temperature variations over the selected period.</p>"
)

widget_box = widgets.VBox([intro_text, start_date, end_date, temp_cutoff, calculate_button, output])
display(widget_box)

